In [ ]:
import numpy as np
import pandas as pd
import pickle
import os
from tqdm import tqdm
import random
from collections import defaultdict
import json
from datetime import datetime

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, 
                             confusion_matrix, classification_report, roc_auc_score)

from scipy import stats
from scipy.fft import fft, fftfreq
from scipy.stats import ttest_rel

import xgboost as xgb

print("="*70)
print("UNIFIED MULTIMODAL PIPELINE - ALL 5 FOLDS")
print("="*70)

# =====================================================
# SET RANDOM SEEDS
# =====================================================

RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)
os.environ['PYTHONHASHSEED'] = str(RANDOM_SEED)

# GPU Configuration
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        tf.config.experimental.enable_op_determinism()
        tf.keras.mixed_precision.set_global_policy('float32')
        print("GPU configured (float32, deterministic mode)")
    except RuntimeError as e:
        print(f"GPU error: {e}")

# =====================================================
# CONFIGURATION 
# =====================================================

EEG_CSV = '/home/jupyter-yin10/EEG_HAR/Overall_Multimodal_pipeline/data/EEG_combined_all.csv'
ACG_CSV = '/home/jupyter-yin10/EEG_HAR/Overall_Multimodal_pipeline/data/ACG_combined.csv'
GYRO_CSV = '/home/jupyter-yin10/EEG_HAR/Overall_Multimodal_pipeline/data/GYRO_combined.csv'
IMPORTANCE_FILE = "/home/jupyter-yin10/EEG_HAR/Overall_Multimodal_pipeline/data/feature_importance_summary.csv"
OUTPUT_DIR = '/home/jupyter-yin10/EEG_HAR/Overall_Multimodal_pipeline/unified_multimodal_5folds'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Window parameters
WINDOW_SIZE_SEC = 4
OVERLAP = 0.0  # 0% overlap

# EEG parameters
EEG_FS = 125
EEG_WINDOW_SAMPLES = int(WINDOW_SIZE_SEC * EEG_FS)  # 500
EEG_CHANNELS = 16

# HAR parameters
HAR_FS = 25
HAR_WINDOW_SAMPLES = int(WINDOW_SIZE_SEC * HAR_FS)  # 100

# Model parameters
D_MODEL = 128
NUM_HEADS = 8
NUM_LAYERS = 2
FF_DIM = 256
EEGNET_DROPOUT = 0.5
TRANSFORMER_DROPOUT = 0.3

# Training parameters
BATCH_SIZE = 32
EPOCHS = 100
LEARNING_RATE = 0.001
PATIENCE = 15

N_FOLDS = 5

print(f"\nConfiguration:")
print(f"  Window: {WINDOW_SIZE_SEC}s, Overlap: {OVERLAP*100:.0f}%")
print(f"  EEG: {EEG_WINDOW_SAMPLES} samples at {EEG_FS}Hz, {EEG_CHANNELS} channels")
print(f"  HAR: {HAR_WINDOW_SAMPLES} samples at {HAR_FS}Hz")
print(f"  Training: All {N_FOLDS} folds")

# =====================================================
# STEP 1: LOAD DATA
# =====================================================

print(f"\n" + "="*70)
print("STEP 1: LOAD RAW DATA")
print("="*70)

eeg_df = pd.read_csv(EEG_CSV)
acg_df = pd.read_csv(ACG_CSV)
gyro_df = pd.read_csv(GYRO_CSV)

print(f"Loaded:")
print(f"  EEG: {eeg_df.shape}")
print(f"  ACG: {acg_df.shape}")
print(f"  GYRO: {gyro_df.shape}")

# Get activity mapping
activity_mapping = eeg_df[['activity_id', 'activity_label']].drop_duplicates().set_index('activity_id')['activity_label'].to_dict()
N_CLASSES = len(activity_mapping)
ACTIVITY_LABELS = [activity_mapping[i] for i in sorted(activity_mapping.keys())]

print(f"\nActivities ({N_CLASSES} classes): {ACTIVITY_LABELS}")

# Save activity mapping
with open(os.path.join(OUTPUT_DIR, 'activity_mapping.json'), 'w') as f:
    json.dump(activity_mapping, f, indent=2)

# =====================================================
# CREATE FEATURE NAME MAPPING
# =====================================================

print(f"\n" + "="*70)
print("CREATING FEATURE NAME MAPPING")
print("="*70)

def create_feature_names():
    """Create feature names matching the importance file convention"""
    feature_names = []
    
    # ACG 3D features (16 features)
    acg_3d_names = ['mag_mean', 'mag_std', 'mag_max', 'mag_min', 
                    'jerk_mean', 'jerk_std', 'jerk_max', 
                    'pitch_mean', 'pitch_std', 'roll_mean', 'roll_std', 
                    'sma', 'corr_12', 'corr_13', 'corr_23', 'energy']
    feature_names.extend([f'acg_3d_{name}' for name in acg_3d_names])
    
    # ACG time-domain (11 features × 3 axes = 33 features)
    time_feats = ['mean', 'std', 'min', 'max', 'median', 'range', 'rms', 
                  'skewness', 'kurtosis', 'zcr', 'mad']
    for axis_num in [1, 2, 3]:
        feature_names.extend([f'acg_axis{axis_num}_time_{feat}' for feat in time_feats])
    
    # ACG frequency-domain (8 features × 3 axes = 24 features)
    freq_feats = ['dominant_freq', 'spectral_energy', 'spectral_entropy', 
                  'mean_freq', 'median_freq', 'power_0_5hz', 'power_5_10hz', 
                  'power_10_12_5hz']
    for axis_num in [1, 2, 3]:
        feature_names.extend([f'acg_axis{axis_num}_freq_{feat}' for feat in freq_feats])
    
    # GYRO 3D features (16 features)
    feature_names.extend([f'gyro_3d_{name}' for name in acg_3d_names])
    
    # GYRO time-domain (11 features × 3 axes = 33 features)
    for axis_num in [1, 2, 3]:
        feature_names.extend([f'gyro_axis{axis_num}_time_{feat}' for feat in time_feats])
    
    # GYRO frequency-domain (8 features × 3 axes = 24 features)
    for axis_num in [1, 2, 3]:
        feature_names.extend([f'gyro_axis{axis_num}_freq_{feat}' for feat in freq_feats])
    
    return feature_names

all_feature_names = create_feature_names()
print(f"Total features in extraction order: {len(all_feature_names)}")

# =====================================================
# STEP 2: CREATE ALIGNED WINDOWS
# =====================================================

print(f"\n" + "="*70)
print("STEP 2: CREATE ALIGNED WINDOWS (0% OVERLAP)")
print("="*70)

# Get unique recordings
eeg_recordings = set(eeg_df.groupby(['subject', 'activity_id']).size().index)
acg_recordings = set(acg_df.groupby(['subject', 'activity_id']).size().index)
gyro_recordings = set(gyro_df.groupby(['subject', 'activity_id']).size().index)

common_recordings = sorted(list(eeg_recordings & acg_recordings & gyro_recordings))

print(f"\nCommon recordings: {len(common_recordings)}")

# Storage for aligned windows
all_eeg_windows = []
all_har_features = []
all_labels = []
all_recording_ids = []

eeg_columns = [f'ch{i}' for i in range(1, 17)]

# HAR feature extraction functions
def extract_time_domain_features(signal):
    features = {}
    features['mean'] = np.mean(signal)
    features['std'] = np.std(signal)
    features['min'] = np.min(signal)
    features['max'] = np.max(signal)
    features['median'] = np.median(signal)
    features['range'] = np.max(signal) - np.min(signal)
    features['rms'] = np.sqrt(np.mean(signal**2))
    features['skewness'] = stats.skew(signal)
    features['kurtosis'] = stats.kurtosis(signal)
    zero_crossings = np.where(np.diff(np.sign(signal)))[0]
    features['zcr'] = len(zero_crossings) / len(signal)
    features['mad'] = np.mean(np.abs(signal - np.mean(signal)))
    return features

def extract_frequency_domain_features(signal, sampling_rate):
    features = {}
    N = len(signal)
    yf = fft(signal)
    xf = fftfreq(N, 1/sampling_rate)
    
    pos_mask = xf > 0
    freqs = xf[pos_mask]
    magnitude = np.abs(yf[pos_mask])
    power = magnitude**2
    
    dominant_idx = np.argmax(magnitude)
    features['dominant_freq'] = freqs[dominant_idx]
    features['spectral_energy'] = np.sum(power)
    
    psd_norm = power / np.sum(power)
    psd_norm = psd_norm[psd_norm > 0]
    features['spectral_entropy'] = -np.sum(psd_norm * np.log2(psd_norm))
    
    features['mean_freq'] = np.sum(freqs * magnitude) / np.sum(magnitude)
    
    cumsum_power = np.cumsum(power)
    median_idx = np.where(cumsum_power >= cumsum_power[-1] / 2)[0][0]
    features['median_freq'] = freqs[median_idx]
    
    band1_mask = (freqs >= 0) & (freqs < 5)
    features['power_0_5hz'] = np.sum(power[band1_mask])
    
    band2_mask = (freqs >= 5) & (freqs < 10)
    features['power_5_10hz'] = np.sum(power[band2_mask])
    
    band3_mask = (freqs >= 10) & (freqs < 12.5)
    features['power_10_12_5hz'] = np.sum(power[band3_mask])
    
    return features

def extract_3d_features(window):
    axis1, axis2, axis3 = window[:, 0], window[:, 1], window[:, 2]
    features = {}
    
    magnitude = np.sqrt(axis1**2 + axis2**2 + axis3**2)
    features['mag_mean'] = np.mean(magnitude)
    features['mag_std'] = np.std(magnitude)
    features['mag_max'] = np.max(magnitude)
    features['mag_min'] = np.min(magnitude)
    
    jerk = np.diff(magnitude)
    features['jerk_mean'] = np.mean(np.abs(jerk))
    features['jerk_std'] = np.std(jerk)
    features['jerk_max'] = np.max(np.abs(jerk))
    
    pitch = np.arctan2(axis2, np.sqrt(axis1**2 + axis3**2))
    roll = np.arctan2(axis1, np.sqrt(axis2**2 + axis3**2))
    
    features['pitch_mean'] = np.mean(pitch)
    features['pitch_std'] = np.std(pitch)
    features['roll_mean'] = np.mean(roll)
    features['roll_std'] = np.std(roll)
    
    features['sma'] = np.sum(np.abs(axis1) + np.abs(axis2) + np.abs(axis3)) / len(axis1)
    
    features['corr_12'] = np.corrcoef(axis1, axis2)[0, 1]
    features['corr_13'] = np.corrcoef(axis1, axis3)[0, 1]
    features['corr_23'] = np.corrcoef(axis2, axis3)[0, 1]
    
    features['energy'] = np.sum(axis1**2 + axis2**2 + axis3**2) / len(axis1)
    
    return features

def extract_all_har_features(acg_window, gyro_window, sampling_rate):
    """Extract all 152 features for HAR in the correct order"""
    all_features = []
    
    # ACG 3D features (16)
    acg_3d = extract_3d_features(acg_window)
    for val in acg_3d.values():
        all_features.append(val)
    
    # ACG time-domain per axis (11 × 3 = 33)
    for axis in range(3):
        time_feats = extract_time_domain_features(acg_window[:, axis])
        for val in time_feats.values():
            all_features.append(val)
    
    # ACG frequency-domain per axis (8 × 3 = 24)
    for axis in range(3):
        freq_feats = extract_frequency_domain_features(acg_window[:, axis], sampling_rate)
        for val in freq_feats.values():
            all_features.append(val)
    
    # GYRO 3D features (16)
    gyro_3d = extract_3d_features(gyro_window)
    for val in gyro_3d.values():
        all_features.append(val)
    
    # GYRO time-domain per axis (11 × 3 = 33)
    for axis in range(3):
        time_feats = extract_time_domain_features(gyro_window[:, axis])
        for val in time_feats.values():
            all_features.append(val)
    
    # GYRO frequency-domain per axis (8 × 3 = 24)
    for axis in range(3):
        freq_feats = extract_frequency_domain_features(gyro_window[:, axis], sampling_rate)
        for val in freq_feats.values():
            all_features.append(val)
    
    return np.array(all_features)

print("\nProcessing recordings and creating aligned windows...")

skipped = 0
processed = 0

for rec_idx, (subject, activity_id) in enumerate(tqdm(common_recordings, desc="Creating windows")):
    # Get EEG data
    eeg_rec = eeg_df[(eeg_df['subject'] == subject) & (eeg_df['activity_id'] == activity_id)]
    eeg_data = eeg_rec[eeg_columns].values
    
    # Get ACG data
    acg_rec = acg_df[(acg_df['subject'] == subject) & (acg_df['activity_id'] == activity_id)].sort_values('timestamp')
    acg_data = acg_rec[['x', 'y', 'z']].values
    
    # Get GYRO data
    gyro_rec = gyro_df[(gyro_df['subject'] == subject) & (gyro_df['activity_id'] == activity_id)].sort_values('timestamp')
    gyro_data = gyro_rec[['x', 'y', 'z']].values
    
    # Calculate time duration for each
    eeg_time = len(eeg_data) / EEG_FS
    acg_time = len(acg_data) / HAR_FS
    gyro_time = len(gyro_data) / HAR_FS
    
    # Use minimum time
    min_time = min(eeg_time, acg_time, gyro_time)
    
    # Skip if too short
    if min_time < WINDOW_SIZE_SEC:
        skipped += 1
        continue
    
    # Truncate all to min_time
    eeg_samples = int(min_time * EEG_FS)
    har_samples = int(min_time * HAR_FS)
    
    eeg_data = eeg_data[:eeg_samples]
    acg_data = acg_data[:har_samples]
    gyro_data = gyro_data[:har_samples]
    
    # Z-score normalize EEG
    eeg_mean = np.mean(eeg_data, axis=0, keepdims=True)
    eeg_std = np.std(eeg_data, axis=0, keepdims=True) + 1e-8
    eeg_data = (eeg_data - eeg_mean) / eeg_std
    
    # Create windows (0% overlap)
    n_windows = int(min_time // WINDOW_SIZE_SEC)
    
    for w_idx in range(n_windows):
        # EEG window
        eeg_start = w_idx * EEG_WINDOW_SAMPLES
        eeg_end = eeg_start + EEG_WINDOW_SAMPLES
        eeg_window = eeg_data[eeg_start:eeg_end, :]  # (500, 16)
        
        # HAR window (ACG + GYRO)
        har_start = w_idx * HAR_WINDOW_SAMPLES
        har_end = har_start + HAR_WINDOW_SAMPLES
        acg_window = acg_data[har_start:har_end, :]  # (100, 3)
        gyro_window = gyro_data[har_start:har_end, :]  # (100, 3)
        
        # Extract HAR features
        har_features = extract_all_har_features(acg_window, gyro_window, HAR_FS)  # (152,)
        
        # Store
        all_eeg_windows.append(eeg_window)
        all_har_features.append(har_features)
        all_labels.append(activity_id)
        all_recording_ids.append(rec_idx)
    
    processed += 1

print(f"\nProcessed: {processed} recordings")
print(f"Skipped: {skipped} recordings (too short)")

# Convert to arrays
X_eeg = np.array(all_eeg_windows)  # (N_windows, 500, 16)
X_har_full = np.array(all_har_features)  # (N_windows, 152)
y = np.array(all_labels)  # (N_windows,)
recording_ids = np.array(all_recording_ids)  # (N_windows,)

print(f"\nAligned dataset:")
print(f"  EEG windows: {X_eeg.shape}")
print(f"  HAR features (full): {X_har_full.shape}")
print(f"  Labels: {y.shape}")
print(f"  Unique labels: {np.unique(y)}")

# =====================================================
# FEATURE SELECTION - CORRECT METHOD
# =====================================================

print(f"\n" + "="*70)
print("FEATURE SELECTION - SELECTING TOP 60 BY IMPORTANCE")
print("="*70)

# Load feature importance
importance_df = pd.read_csv(IMPORTANCE_FILE)
print(f"\nLoaded importance file: {importance_df.shape}")

# Get top 60 feature names
top_60_feature_names = importance_df.head(60)['feature'].tolist()
print(f"\nTop 10 most important features:")
for i, name in enumerate(top_60_feature_names[:10], 1):
    print(f"  {i}. {name}")

# Create index mapping
print(f"\nMapping feature names to indices...")
top_60_indices = []
missing_features = []

for feat_name in top_60_feature_names:
    if feat_name in all_feature_names:
        idx = all_feature_names.index(feat_name)
        top_60_indices.append(idx)
    else:
        missing_features.append(feat_name)

if missing_features:
    print(f"\nWARNING: {len(missing_features)} features not found!")
    raise ValueError("Feature name mismatch - cannot proceed")
else:
    print(f"\nSuccessfully mapped all 60 features!")

# Select the top 60 features
X_har = X_har_full[:, top_60_indices]

print(f"\nHAR features (top 60 by importance): {X_har.shape}")

# =====================================================
# STEP 3: CREATE STRATIFIED 5-FOLD SPLIT
# =====================================================

print(f"\n" + "="*70)
print("STEP 3: CREATE STRATIFIED 5-FOLD SPLIT (WINDOW-LEVEL)")
print("="*70)

skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_SEED)

fold_splits = []
for fold_idx, (train_idx, test_idx) in enumerate(skf.split(X_eeg, y)):
    fold_splits.append({
        'fold': fold_idx + 1,
        'train_idx': train_idx,
        'test_idx': test_idx
    })
    print(f"  Fold {fold_idx + 1}: Train={len(train_idx)}, Test={len(test_idx)}")

print(f"\nNote: Using window-level stratified CV with 0% overlap and per-recording normalization")

# =====================================================
# HELPER FUNCTIONS FOR METRICS
# =====================================================

def calculate_per_class_metrics(y_true, y_pred, y_pred_proba, class_names):
    """Calculate comprehensive per-class metrics"""
    
    # Per-class precision, recall, F1
    precision_per_class = precision_score(y_true, y_pred, average=None, zero_division=0)
    recall_per_class = recall_score(y_true, y_pred, average=None, zero_division=0)
    f1_per_class = f1_score(y_true, y_pred, average=None, zero_division=0)
    
    # Per-class accuracy (fraction of correct predictions for each class)
    accuracy_per_class = []
    for class_id in range(len(class_names)):
        class_mask = (y_true == class_id + 1)
        if np.sum(class_mask) > 0:
            class_acc = np.sum((y_pred == class_id + 1) & class_mask) / np.sum(class_mask)
        else:
            class_acc = 0.0
        accuracy_per_class.append(class_acc)
    
    # Per-class AUC (one-vs-rest)
    auc_per_class = []
    for class_id in range(len(class_names)):
        try:
            y_true_binary = (y_true == class_id + 1).astype(int)
            y_score = y_pred_proba[:, class_id]
            if len(np.unique(y_true_binary)) > 1:  # Need both classes present
                auc = roc_auc_score(y_true_binary, y_score)
            else:
                auc = 0.0
        except:
            auc = 0.0
        auc_per_class.append(auc)
    
    # Create detailed dictionary
    per_class_metrics = {}
    for i, class_name in enumerate(class_names):
        per_class_metrics[class_name] = {
            'accuracy': float(accuracy_per_class[i]),
            'precision': float(precision_per_class[i]),
            'recall': float(recall_per_class[i]),
            'f1': float(f1_per_class[i]),
            'auc': float(auc_per_class[i])
        }
    
    return per_class_metrics

def print_per_class_metrics(per_class_metrics, title="PER-CLASS METRICS"):
    """Pretty print per-class metrics"""
    print(f"\n{title}:")
    print(f"{'Class':<30s} {'Acc':>8s} {'Prec':>8s} {'Rec':>8s} {'F1':>8s} {'AUC':>8s}")
    print("-" * 75)
    for class_name, metrics in per_class_metrics.items():
        print(f"{class_name:<30s} "
              f"{metrics['accuracy']*100:>7.2f}% "
              f"{metrics['precision']:>7.4f} "
              f"{metrics['recall']:>7.4f} "
              f"{metrics['f1']:>7.4f} "
              f"{metrics['auc']:>7.4f}")

def compute_robustness_metrics(per_class_metrics):
    """Compute worst-task accuracy and cross-task variance"""
    class_accuracies = [per_class_metrics[class_name]['accuracy'] for class_name in ACTIVITY_LABELS]
    
    worst_task_acc = min(class_accuracies)
    best_task_acc = max(class_accuracies)
    cross_task_variance = np.var(class_accuracies)
    cross_task_std = np.std(class_accuracies)
    
    return {
        'worst_task_accuracy': float(worst_task_acc),
        'best_task_accuracy': float(best_task_acc),
        'cross_task_variance': float(cross_task_variance),
        'cross_task_std': float(cross_task_std)
    }

# =====================================================
# STEP 4: TRAIN ALL 5 FOLDS
# =====================================================

# Store all results
all_fold_results = []

# EEGNet + Transformer model builder
def build_eegnet_transformer():
    class TransformerBlock(layers.Layer):
        def __init__(self, d_model, num_heads, ff_dim, dropout_rate=0.1, **kwargs):
            super(TransformerBlock, self).__init__(**kwargs)
            self.d_model = d_model
            self.num_heads = num_heads
            self.ff_dim = ff_dim
            self.dropout_rate = dropout_rate
            
            self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads, dropout=dropout_rate)
            self.ffn = keras.Sequential([layers.Dense(ff_dim, activation="relu"), layers.Dense(d_model)])
            self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
            self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
            self.dropout1 = layers.Dropout(dropout_rate)
            self.dropout2 = layers.Dropout(dropout_rate)
        
        def call(self, inputs, training=False):
            attn_output = self.att(inputs, inputs, training=training)
            attn_output = self.dropout1(attn_output, training=training)
            out1 = self.layernorm1(inputs + attn_output)
            
            ffn_output = self.ffn(out1)
            ffn_output = self.dropout2(ffn_output, training=training)
            return self.layernorm2(out1 + ffn_output)
        
        def get_config(self):
            config = super().get_config()
            config.update({'d_model': self.d_model, 'num_heads': self.num_heads, 
                          'ff_dim': self.ff_dim, 'dropout_rate': self.dropout_rate})
            return config

    class PositionalEncoding(layers.Layer):
        def __init__(self, sequence_length, d_model, **kwargs):
            super(PositionalEncoding, self).__init__(**kwargs)
            self.sequence_length = sequence_length
            self.d_model = d_model
            self.pos_encoding = self.add_weight(name='pos_encoding', shape=(1, sequence_length, d_model),
                                               initializer='zeros', trainable=True)
        
        def call(self, x):
            return x + self.pos_encoding
        
        def get_config(self):
            config = super().get_config()
            config.update({'sequence_length': self.sequence_length, 'd_model': self.d_model})
            return config

    F1, D, F2 = 8, 2, 16
    kernel_length = 64

    input_layer = layers.Input(shape=(EEG_WINDOW_SAMPLES, EEG_CHANNELS, 1))

    block1 = layers.Conv2D(F1, (kernel_length, 1), padding='same', use_bias=False)(input_layer)
    block1 = layers.BatchNormalization()(block1)
    block1 = layers.DepthwiseConv2D((1, EEG_CHANNELS), use_bias=False, depth_multiplier=D,
                                   depthwise_constraint=keras.constraints.max_norm(1.))(block1)
    block1 = layers.BatchNormalization()(block1)
    block1 = layers.Activation('elu')(block1)
    block1 = layers.AveragePooling2D((4, 1))(block1)
    block1 = layers.Dropout(EEGNET_DROPOUT)(block1)

    block2 = layers.SeparableConv2D(F2, (16, 1), use_bias=False, padding='same')(block1)
    block2 = layers.BatchNormalization()(block2)
    block2 = layers.Activation('elu')(block2)
    block2 = layers.AveragePooling2D((8, 1))(block2)
    block2 = layers.Dropout(EEGNET_DROPOUT)(block2)

    reshape = layers.Reshape((15, 16))(block2)
    projection = layers.Dense(D_MODEL)(reshape)
    pos_encoding = PositionalEncoding(sequence_length=15, d_model=D_MODEL)(projection)

    x = pos_encoding
    for i in range(NUM_LAYERS):
        x = TransformerBlock(d_model=D_MODEL, num_heads=NUM_HEADS, ff_dim=FF_DIM,
                            dropout_rate=TRANSFORMER_DROPOUT, name=f'transformer_block_{i+1}')(x)

    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(TRANSFORMER_DROPOUT)(x)
    dense = layers.Dense(N_CLASSES, kernel_constraint=keras.constraints.max_norm(0.25))(x)
    softmax = layers.Activation('softmax')(dense)

    model = models.Model(inputs=input_layer, outputs=softmax)
    
    return model

for fold_idx in range(N_FOLDS):
    
    print(f"\n" + "="*70)
    print(f"FOLD {fold_idx + 1}/{N_FOLDS}")
    print("="*70)
    
    fold_data = fold_splits[fold_idx]
    train_idx = fold_data['train_idx']
    test_idx = fold_data['test_idx']
    
    # Split train into train/val (80/20)
    train_idx_actual, val_idx = train_test_split(
        train_idx,
        test_size=0.2,
        random_state=RANDOM_SEED,
        stratify=y[train_idx]
    )
    
    print(f"\nData split:")
    print(f"  Train: {len(train_idx_actual)} windows")
    print(f"  Val:   {len(val_idx)} windows")
    print(f"  Test:  {len(test_idx)} windows")
    
    # ==========================================
    # EEG BRANCH: EEGNet + Transformer
    # ==========================================
    
    print(f"\n" + "-"*70)
    print("EEG BRANCH: EEGNET + TRANSFORMER")
    print("-"*70)
    
    # Prepare data
    X_eeg_train = X_eeg[train_idx_actual]
    X_eeg_val = X_eeg[val_idx]
    X_eeg_test = X_eeg[test_idx]
    
    y_eeg_train = y[train_idx_actual]
    y_eeg_val = y[val_idx]
    y_eeg_test = y[test_idx]
    
    # Reshape for model
    X_eeg_train_reshaped = X_eeg_train.reshape(len(X_eeg_train), EEG_WINDOW_SAMPLES, EEG_CHANNELS, 1)
    X_eeg_val_reshaped = X_eeg_val.reshape(len(X_eeg_val), EEG_WINDOW_SAMPLES, EEG_CHANNELS, 1)
    X_eeg_test_reshaped = X_eeg_test.reshape(len(X_eeg_test), EEG_WINDOW_SAMPLES, EEG_CHANNELS, 1)
    
    # Convert labels to categorical
    y_eeg_train_cat = to_categorical(y_eeg_train - 1, N_CLASSES)
    y_eeg_val_cat = to_categorical(y_eeg_val - 1, N_CLASSES)
    
    # Build model
    print("\nBuilding EEGNet + Transformer...")
    eeg_model = build_eegnet_transformer()
    
    # Compile
    eeg_model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Class weights
    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_eeg_train), y=y_eeg_train)
    class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}
    
    # Callbacks
    early_stop = EarlyStopping(monitor='val_loss', patience=PATIENCE, restore_best_weights=True, verbose=0)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7, verbose=0)
    
    # Train
    print("Training EEG model...")
    eeg_history = eeg_model.fit(
        X_eeg_train_reshaped, y_eeg_train_cat,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(X_eeg_val_reshaped, y_eeg_val_cat),
        class_weight=class_weight_dict,
        callbacks=[early_stop, reduce_lr],
        verbose=0
    )
    
    print(f"Training complete! Epochs: {len(eeg_history.history['loss'])}")
    
    # Predict on test
    print("Evaluating EEG on test set...")
    eeg_test_probs = eeg_model.predict(X_eeg_test_reshaped, verbose=0)
    eeg_test_preds = np.argmax(eeg_test_probs, axis=1) + 1
    
    eeg_test_acc = accuracy_score(y_eeg_test, eeg_test_preds)
    eeg_test_f1 = f1_score(y_eeg_test, eeg_test_preds, average='macro')
    
    print(f"EEG Test Results: Accuracy={eeg_test_acc*100:.2f}%, F1={eeg_test_f1:.4f}")
    
    # Calculate per-class metrics for EEG
    eeg_per_class = calculate_per_class_metrics(y_eeg_test, eeg_test_preds, eeg_test_probs, ACTIVITY_LABELS)
    print_per_class_metrics(eeg_per_class, "EEG PER-CLASS METRICS")
    
    # ==========================================
    # HAR BRANCH: XGBoost
    # ==========================================
    
    print(f"\n" + "-"*70)
    print("HAR BRANCH: XGBOOST")
    print("-"*70)
    
    # Prepare data
    X_har_train = X_har[train_idx_actual]
    X_har_val = X_har[val_idx]
    X_har_test = X_har[test_idx]
    
    y_har_train = y[train_idx_actual]
    y_har_val = y[val_idx]
    y_har_test = y[test_idx]
    
    # Normalize
    scaler = StandardScaler()
    X_har_train_scaled = scaler.fit_transform(X_har_train)
    X_har_val_scaled = scaler.transform(X_har_val)
    X_har_test_scaled = scaler.transform(X_har_test)
    
    # Train XGBoost
    print("Training HAR model...")
    har_model = xgb.XGBClassifier(
        objective='multi:softmax',
        num_class=N_CLASSES,
        max_depth=6,
        learning_rate=0.1,
        n_estimators=100,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=RANDOM_SEED,
        tree_method='gpu_hist',
        gpu_id=0,
        eval_metric='mlogloss'
    )
    
    har_model.fit(X_har_train_scaled, y_har_train - 1, verbose=False)
    
    print("Training complete!")
    
    # Predict on test
    print("Evaluating HAR on test set...")
    har_test_probs = har_model.predict_proba(X_har_test_scaled)
    har_test_preds = har_model.predict(X_har_test_scaled) + 1 
    
    har_test_acc = accuracy_score(y_har_test, har_test_preds)
    har_test_f1 = f1_score(y_har_test, har_test_preds, average='macro')
    
    print(f"HAR Test Results: Accuracy={har_test_acc*100:.2f}%, F1={har_test_f1:.4f}")
    
    # Calculate per-class metrics for HAR
    har_per_class = calculate_per_class_metrics(y_har_test, har_test_preds, har_test_probs, ACTIVITY_LABELS)
    print_per_class_metrics(har_per_class, "HAR PER-CLASS METRICS")
    
    # ==========================================
    # FUSION
    # ==========================================
    
    print(f"\n" + "-"*70)
    print("FUSION METHODS")
    print("-"*70)
    
    # Method 1: Simple Averaging
    print("\n1. Simple Averaging (50-50)...")
    simple_avg_probs = 0.5 * eeg_test_probs + 0.5 * har_test_probs
    simple_avg_preds = np.argmax(simple_avg_probs, axis=1) + 1
    simple_avg_acc = accuracy_score(y_eeg_test, simple_avg_preds)
    simple_avg_f1 = f1_score(y_eeg_test, simple_avg_preds, average='macro')
    
    print(f"   Accuracy: {simple_avg_acc*100:.2f}%, F1: {simple_avg_f1:.4f}")
    simple_avg_per_class = calculate_per_class_metrics(y_eeg_test, simple_avg_preds, simple_avg_probs, ACTIVITY_LABELS)
    
    # Method 2: Weighted Averaging
    print("\n2. Weighted Averaging (EEG: 0.503, HAR: 0.497)...")
    weight_eeg = 0.503
    weight_har = 0.497
    weighted_avg_probs = weight_eeg * eeg_test_probs + weight_har * har_test_probs
    weighted_avg_preds = np.argmax(weighted_avg_probs, axis=1) + 1
    weighted_avg_acc = accuracy_score(y_eeg_test, weighted_avg_preds)
    weighted_avg_f1 = f1_score(y_eeg_test, weighted_avg_preds, average='macro')
    
    print(f"   Accuracy: {weighted_avg_acc*100:.2f}%, F1: {weighted_avg_f1:.4f}")
    weighted_avg_per_class = calculate_per_class_metrics(y_eeg_test, weighted_avg_preds, weighted_avg_probs, ACTIVITY_LABELS)
    
    # Method 3: Logistic Regression
    print("\n3. Logistic Regression...")
    eeg_val_probs = eeg_model.predict(X_eeg_val_reshaped, verbose=0)
    har_val_probs = har_model.predict_proba(X_har_val_scaled)
    
    X_val_fusion = np.hstack([eeg_val_probs, har_val_probs])
    X_test_fusion = np.hstack([eeg_test_probs, har_test_probs])
    
    lr_fusion = LogisticRegression(max_iter=1000, random_state=RANDOM_SEED)
    lr_fusion.fit(X_val_fusion, y_eeg_val)
    
    lr_preds = lr_fusion.predict(X_test_fusion)
    lr_probs = lr_fusion.predict_proba(X_test_fusion)
    lr_acc = accuracy_score(y_eeg_test, lr_preds)
    lr_f1 = f1_score(y_eeg_test, lr_preds, average='macro')
    
    print(f"   Accuracy: {lr_acc*100:.2f}%, F1: {lr_f1:.4f}")
    lr_per_class = calculate_per_class_metrics(y_eeg_test, lr_preds, lr_probs, ACTIVITY_LABELS)
    
    # Method 4: MLP
    print("\n4. MLP Fusion...")
    mlp_fusion = MLPClassifier(
        hidden_layer_sizes=(64, 32),
        activation='relu',
        solver='adam',
        max_iter=500,
        random_state=RANDOM_SEED,
        early_stopping=True,
        validation_fraction=0.1
    )
    mlp_fusion.fit(X_val_fusion, y_eeg_val)
    
    mlp_preds = mlp_fusion.predict(X_test_fusion)
    mlp_probs = mlp_fusion.predict_proba(X_test_fusion)
    mlp_acc = accuracy_score(y_eeg_test, mlp_preds)
    mlp_f1 = f1_score(y_eeg_test, mlp_preds, average='macro')
    
    print(f"   Accuracy: {mlp_acc*100:.2f}%, F1: {mlp_f1:.4f}")
    mlp_per_class = calculate_per_class_metrics(y_eeg_test, mlp_preds, mlp_probs, ACTIVITY_LABELS)
    
    # ==========================================
    # COMPUTE ROBUSTNESS METRICS FOR THIS FOLD
    # ==========================================
    
    print(f"\n" + "-"*70)
    print("ROBUSTNESS METRICS")
    print("-"*70)
    
    # Compute for each method
    eeg_robustness = compute_robustness_metrics(eeg_per_class)
    har_robustness = compute_robustness_metrics(har_per_class)
    simple_avg_robustness = compute_robustness_metrics(simple_avg_per_class)
    weighted_avg_robustness = compute_robustness_metrics(weighted_avg_per_class)
    lr_robustness = compute_robustness_metrics(lr_per_class)
    mlp_robustness = compute_robustness_metrics(mlp_per_class)
    
    # Print robustness metrics
    print(f"\n{'Method':<25s} {'Worst-Task':>12s} {'Best-Task':>12s} {'Cross-Task Std':>15s}")
    print("-" * 70)
    print(f"{'EEG only':<25s} {eeg_robustness['worst_task_accuracy']*100:>11.2f}% {eeg_robustness['best_task_accuracy']*100:>11.2f}% {eeg_robustness['cross_task_std']*100:>14.2f}%")
    print(f"{'HAR only':<25s} {har_robustness['worst_task_accuracy']*100:>11.2f}% {har_robustness['best_task_accuracy']*100:>11.2f}% {har_robustness['cross_task_std']*100:>14.2f}%")
    print(f"{'Simple Averaging':<25s} {simple_avg_robustness['worst_task_accuracy']*100:>11.2f}% {simple_avg_robustness['best_task_accuracy']*100:>11.2f}% {simple_avg_robustness['cross_task_std']*100:>14.2f}%")
    print(f"{'Weighted Averaging':<25s} {weighted_avg_robustness['worst_task_accuracy']*100:>11.2f}% {weighted_avg_robustness['best_task_accuracy']*100:>11.2f}% {weighted_avg_robustness['cross_task_std']*100:>14.2f}%")
    print(f"{'Logistic Regression':<25s} {lr_robustness['worst_task_accuracy']*100:>11.2f}% {lr_robustness['best_task_accuracy']*100:>11.2f}% {lr_robustness['cross_task_std']*100:>14.2f}%")
    print(f"{'MLP':<25s} {mlp_robustness['worst_task_accuracy']*100:>11.2f}% {mlp_robustness['best_task_accuracy']*100:>11.2f}% {mlp_robustness['cross_task_std']*100:>14.2f}%")
    
    # ==========================================
    # FOLD SUMMARY
    # ==========================================
    
    print(f"\n" + "="*70)
    print(f"FOLD {fold_idx + 1} SUMMARY")
    print("="*70)
    
    print(f"\n{'Method':<25s} {'Accuracy':>12s} {'F1 (macro)':>12s}")
    print("-" * 52)
    print(f"{'EEG only':<25s} {eeg_test_acc*100:>11.2f}% {eeg_test_f1:>12.4f}")
    print(f"{'HAR only':<25s} {har_test_acc*100:>11.2f}% {har_test_f1:>12.4f}")
    print(f"{'Simple Averaging':<25s} {simple_avg_acc*100:>11.2f}% {simple_avg_f1:>12.4f}")
    print(f"{'Weighted Averaging':<25s} {weighted_avg_acc*100:>11.2f}% {weighted_avg_f1:>12.4f}")
    print(f"{'Logistic Regression':<25s} {lr_acc*100:>11.2f}% {lr_f1:>12.4f}")
    print(f"{'MLP':<25s} {mlp_acc*100:>11.2f}% {mlp_f1:>12.4f}")
    
    # Store fold results WITH robustness metrics
    fold_results = {
        'fold': fold_idx + 1,
        'eeg': {
            'accuracy': float(eeg_test_acc),
            'f1_macro': float(eeg_test_f1),
            'per_class': eeg_per_class,
            'worst_task_accuracy': eeg_robustness['worst_task_accuracy'],
            'best_task_accuracy': eeg_robustness['best_task_accuracy'],
            'cross_task_variance': eeg_robustness['cross_task_variance'],
            'cross_task_std': eeg_robustness['cross_task_std']
        },
        'har': {
            'accuracy': float(har_test_acc),
            'f1_macro': float(har_test_f1),
            'per_class': har_per_class,
            'worst_task_accuracy': har_robustness['worst_task_accuracy'],
            'best_task_accuracy': har_robustness['best_task_accuracy'],
            'cross_task_variance': har_robustness['cross_task_variance'],
            'cross_task_std': har_robustness['cross_task_std']
        },
        'simple_averaging': {
            'accuracy': float(simple_avg_acc),
            'f1_macro': float(simple_avg_f1),
            'per_class': simple_avg_per_class,
            'worst_task_accuracy': simple_avg_robustness['worst_task_accuracy'],
            'best_task_accuracy': simple_avg_robustness['best_task_accuracy'],
            'cross_task_variance': simple_avg_robustness['cross_task_variance'],
            'cross_task_std': simple_avg_robustness['cross_task_std']
        },
        'weighted_averaging': {
            'accuracy': float(weighted_avg_acc),
            'f1_macro': float(weighted_avg_f1),
            'per_class': weighted_avg_per_class,
            'worst_task_accuracy': weighted_avg_robustness['worst_task_accuracy'],
            'best_task_accuracy': weighted_avg_robustness['best_task_accuracy'],
            'cross_task_variance': weighted_avg_robustness['cross_task_variance'],
            'cross_task_std': weighted_avg_robustness['cross_task_std']
        },
        'logistic_regression': {
            'accuracy': float(lr_acc),
            'f1_macro': float(lr_f1),
            'per_class': lr_per_class,
            'worst_task_accuracy': lr_robustness['worst_task_accuracy'],
            'best_task_accuracy': lr_robustness['best_task_accuracy'],
            'cross_task_variance': lr_robustness['cross_task_variance'],
            'cross_task_std': lr_robustness['cross_task_std']
        },
        'mlp': {
            'accuracy': float(mlp_acc),
            'f1_macro': float(mlp_f1),
            'per_class': mlp_per_class,
            'worst_task_accuracy': mlp_robustness['worst_task_accuracy'],
            'best_task_accuracy': mlp_robustness['best_task_accuracy'],
            'cross_task_variance': mlp_robustness['cross_task_variance'],
            'cross_task_std': mlp_robustness['cross_task_std']
        }
    }
    
    all_fold_results.append(fold_results)
    
    # Save fold results
    fold_dir = os.path.join(OUTPUT_DIR, f'fold_{fold_idx + 1}')
    os.makedirs(fold_dir, exist_ok=True)
    
    with open(os.path.join(fold_dir, 'results.json'), 'w') as f:
        json.dump(fold_results, f, indent=2)
    
    # Save confusion matrices
    np.save(os.path.join(fold_dir, 'cm_eeg.npy'), confusion_matrix(y_eeg_test, eeg_test_preds))
    np.save(os.path.join(fold_dir, 'cm_har.npy'), confusion_matrix(y_har_test, har_test_preds))
    np.save(os.path.join(fold_dir, 'cm_simple_avg.npy'), confusion_matrix(y_eeg_test, simple_avg_preds))
    np.save(os.path.join(fold_dir, 'cm_weighted_avg.npy'), confusion_matrix(y_eeg_test, weighted_avg_preds))
    np.save(os.path.join(fold_dir, 'cm_lr.npy'), confusion_matrix(y_eeg_test, lr_preds))
    np.save(os.path.join(fold_dir, 'cm_mlp.npy'), confusion_matrix(y_eeg_test, mlp_preds))
    
    # Clear models to free memory
    del eeg_model
    tf.keras.backend.clear_session()

# =====================================================
# AGGREGATE RESULTS ACROSS ALL FOLDS
# =====================================================

print(f"\n" + "="*70)
print("FINAL RESULTS - AGGREGATE ACROSS ALL 5 FOLDS")
print("="*70)

# Extract metrics for each method
methods = ['eeg', 'har', 'simple_averaging', 'weighted_averaging', 'logistic_regression', 'mlp']
method_names = {
    'eeg': 'EEG only',
    'har': 'HAR only',
    'simple_averaging': 'Simple Averaging',
    'weighted_averaging': 'Weighted Averaging',
    'logistic_regression': 'Logistic Regression',
    'mlp': 'MLP'
}

aggregate_results = {}

for method in methods:
    accuracies = [r[method]['accuracy'] for r in all_fold_results]
    f1_scores = [r[method]['f1_macro'] for r in all_fold_results]
    
    # Collect robustness metrics
    worst_task_accs = [r[method]['worst_task_accuracy'] for r in all_fold_results]
    best_task_accs = [r[method]['best_task_accuracy'] for r in all_fold_results]
    cross_task_variances = [r[method]['cross_task_variance'] for r in all_fold_results]
    cross_task_stds = [r[method]['cross_task_std'] for r in all_fold_results]
    
    # Aggregate per-class metrics
    per_class_aggregate = {}
    for class_name in ACTIVITY_LABELS:
        class_accuracies = [r[method]['per_class'][class_name]['accuracy'] for r in all_fold_results]
        class_precisions = [r[method]['per_class'][class_name]['precision'] for r in all_fold_results]
        class_recalls = [r[method]['per_class'][class_name]['recall'] for r in all_fold_results]
        class_f1s = [r[method]['per_class'][class_name]['f1'] for r in all_fold_results]
        class_aucs = [r[method]['per_class'][class_name]['auc'] for r in all_fold_results]
        
        per_class_aggregate[class_name] = {
            'accuracy_mean': float(np.mean(class_accuracies)),
            'accuracy_std': float(np.std(class_accuracies)),
            'precision_mean': float(np.mean(class_precisions)),
            'precision_std': float(np.std(class_precisions)),
            'recall_mean': float(np.mean(class_recalls)),
            'recall_std': float(np.std(class_recalls)),
            'f1_mean': float(np.mean(class_f1s)),
            'f1_std': float(np.std(class_f1s)),
            'auc_mean': float(np.mean(class_aucs)),
            'auc_std': float(np.std(class_aucs))
        }
    
    aggregate_results[method] = {
        'accuracy_mean': float(np.mean(accuracies)),
        'accuracy_std': float(np.std(accuracies)),
        'f1_macro_mean': float(np.mean(f1_scores)),
        'f1_macro_std': float(np.std(f1_scores)),
        'worst_task_accuracy_mean': float(np.mean(worst_task_accs)),
        'worst_task_accuracy_std': float(np.std(worst_task_accs)),
        'best_task_accuracy_mean': float(np.mean(best_task_accs)),
        'best_task_accuracy_std': float(np.std(best_task_accs)),
        'cross_task_variance_mean': float(np.mean(cross_task_variances)),
        'cross_task_variance_std': float(np.std(cross_task_variances)),
        'cross_task_std_mean': float(np.mean(cross_task_stds)),
        'cross_task_std_std': float(np.std(cross_task_stds)),
        'per_class': per_class_aggregate
    }

# =====================================================
# STATISTICAL TESTS: PAIRED T-TESTS ACROSS FOLDS
# =====================================================

print(f"\n" + "="*70)
print("STATISTICAL SIGNIFICANCE TESTS (PAIRED T-TESTS)")
print("="*70)

# Collect fold-level metrics for each method
method_accuracies = {method: [] for method in methods}
method_f1s = {method: [] for method in methods}
method_worst_task = {method: [] for method in methods}

for fold_result in all_fold_results:
    for method in methods:
        method_accuracies[method].append(fold_result[method]['accuracy'])
        method_f1s[method].append(fold_result[method]['f1_macro'])
        method_worst_task[method].append(fold_result[method]['worst_task_accuracy'])

# Convert to arrays
for method in methods:
    method_accuracies[method] = np.array(method_accuracies[method])
    method_f1s[method] = np.array(method_f1s[method])
    method_worst_task[method] = np.array(method_worst_task[method])

# Test 1: Fusion (Logistic Regression) vs EEG
print("\n1. Logistic Regression Fusion vs EEG-only:")
t_stat_acc, p_value_acc = ttest_rel(method_accuracies['logistic_regression'], method_accuracies['eeg'])
t_stat_f1, p_value_f1 = ttest_rel(method_f1s['logistic_regression'], method_f1s['eeg'])
t_stat_worst, p_value_worst = ttest_rel(method_worst_task['logistic_regression'], method_worst_task['eeg'])

print(f"   Overall Accuracy:  t={t_stat_acc:.4f}, p={p_value_acc:.6f} {'***' if p_value_acc < 0.001 else '**' if p_value_acc < 0.01 else '*' if p_value_acc < 0.05 else 'ns'}")
print(f"   Macro F1-score:    t={t_stat_f1:.4f}, p={p_value_f1:.6f} {'***' if p_value_f1 < 0.001 else '**' if p_value_f1 < 0.01 else '*' if p_value_f1 < 0.05 else 'ns'}")
print(f"   Worst-task Acc:    t={t_stat_worst:.4f}, p={p_value_worst:.6f} {'***' if p_value_worst < 0.001 else '**' if p_value_worst < 0.01 else '*' if p_value_worst < 0.05 else 'ns'}")

# Test 2: Fusion (Logistic Regression) vs HAR
print("\n2. Logistic Regression Fusion vs HAR-only:")
t_stat_acc2, p_value_acc2 = ttest_rel(method_accuracies['logistic_regression'], method_accuracies['har'])
t_stat_f12, p_value_f12 = ttest_rel(method_f1s['logistic_regression'], method_f1s['har'])
t_stat_worst2, p_value_worst2 = ttest_rel(method_worst_task['logistic_regression'], method_worst_task['har'])

print(f"   Overall Accuracy:  t={t_stat_acc2:.4f}, p={p_value_acc2:.6f} {'***' if p_value_acc2 < 0.001 else '**' if p_value_acc2 < 0.01 else '*' if p_value_acc2 < 0.05 else 'ns'}")
print(f"   Macro F1-score:    t={t_stat_f12:.4f}, p={p_value_f12:.6f} {'***' if p_value_f12 < 0.001 else '**' if p_value_f12 < 0.01 else '*' if p_value_f12 < 0.05 else 'ns'}")
print(f"   Worst-task Acc:    t={t_stat_worst2:.4f}, p={p_value_worst2:.6f} {'***' if p_value_worst2 < 0.001 else '**' if p_value_worst2 < 0.01 else '*' if p_value_worst2 < 0.05 else 'ns'}")

# Test 3: Fusion vs Best Single Modality (per fold)
print("\n3. Logistic Regression Fusion vs Best Single Modality (per fold):")
best_single_modality_per_fold = []
for i in range(N_FOLDS):
    best_acc = max(method_accuracies['eeg'][i], method_accuracies['har'][i])
    best_single_modality_per_fold.append(best_acc)

best_single_modality_per_fold = np.array(best_single_modality_per_fold)
t_stat3, p_value3 = ttest_rel(method_accuracies['logistic_regression'], best_single_modality_per_fold)

print(f"   Overall Accuracy:  t={t_stat3:.4f}, p={p_value3:.6f} {'***' if p_value3 < 0.001 else '**' if p_value3 < 0.01 else '*' if p_value3 < 0.05 else 'ns'}")

# Store statistical test results
statistical_tests = {
    'fusion_vs_eeg': {
        'accuracy': {'t_statistic': float(t_stat_acc), 'p_value': float(p_value_acc)},
        'f1_macro': {'t_statistic': float(t_stat_f1), 'p_value': float(p_value_f1)},
        'worst_task': {'t_statistic': float(t_stat_worst), 'p_value': float(p_value_worst)}
    },
    'fusion_vs_har': {
        'accuracy': {'t_statistic': float(t_stat_acc2), 'p_value': float(p_value_acc2)},
        'f1_macro': {'t_statistic': float(t_stat_f12), 'p_value': float(p_value_f12)},
        'worst_task': {'t_statistic': float(t_stat_worst2), 'p_value': float(p_value_worst2)}
    },
    'fusion_vs_best_single': {
        'accuracy': {'t_statistic': float(t_stat3), 'p_value': float(p_value3)}
    }
}

print("\n Statistical significance tests complete")
print(f"  (* p<0.05, ** p<0.01, *** p<0.001, ns = not significant)")

# Print overall summary
print(f"\n{'Method':<25s} {'Accuracy':>20s} {'F1 (macro)':>20s} {'Worst-Task Acc':>20s}")
print("-" * 90)
for method in methods:
    acc_mean = aggregate_results[method]['accuracy_mean']
    acc_std = aggregate_results[method]['accuracy_std']
    f1_mean = aggregate_results[method]['f1_macro_mean']
    f1_std = aggregate_results[method]['f1_macro_std']
    worst_mean = aggregate_results[method]['worst_task_accuracy_mean']
    worst_std = aggregate_results[method]['worst_task_accuracy_std']
    
    print(f"{method_names[method]:<25s} "
          f"{acc_mean*100:>6.2f}% ± {acc_std*100:>4.2f}%  "
          f"{f1_mean:>6.4f} ± {f1_std:>5.4f}  "
          f"{worst_mean*100:>6.2f}% ± {worst_std*100:>4.2f}%")

# Add robustness metrics summary
print(f"\n" + "="*70)
print("ROBUSTNESS METRICS SUMMARY")
print("="*70)

print(f"\n{'Method':<25s} {'Worst-Task Acc':>20s} {'Cross-Task Std':>20s}")
print("-" * 68)
for method in methods:
    worst_mean = aggregate_results[method]['worst_task_accuracy_mean']
    worst_std = aggregate_results[method]['worst_task_accuracy_std']
    std_mean = aggregate_results[method]['cross_task_std_mean']
    std_std = aggregate_results[method]['cross_task_std_std']
    
    print(f"{method_names[method]:<25s} "
          f"{worst_mean*100:>6.2f}% ± {worst_std*100:>4.2f}%  "
          f"{std_mean*100:>6.2f}% ± {std_std*100:>4.2f}%")

print(f"\nInterpretation:")
print(f"  - Worst-task accuracy: Minimum per-class accuracy (measures coverage)")
print(f"  - Cross-task std: Standard deviation of per-class accuracies (measures consistency)")
print(f"  - Lower cross-task std = more consistent across activities")

print(f"\n" + "="*70)
print("PER-CLASS RESULTS FOR EEG-ONLY")
print("="*70)

print(f"\n{'Class':<30s} {'Acc':>12s} {'Prec':>12s} {'Rec':>12s} {'F1':>12s} {'AUC':>12s}")
print("-" * 95)

for class_name in ACTIVITY_LABELS:
    metrics = aggregate_results['eeg']['per_class'][class_name]
    print(f"{class_name:<30s} "
          f"{metrics['accuracy_mean']*100:>5.2f}±{metrics['accuracy_std']*100:>4.2f}% "
          f"{metrics['precision_mean']:>5.3f}±{metrics['precision_std']:>4.3f} "
          f"{metrics['recall_mean']:>5.3f}±{metrics['recall_std']:>4.3f} "
          f"{metrics['f1_mean']:>5.3f}±{metrics['f1_std']:>4.3f} "
          f"{metrics['auc_mean']:>5.3f}±{metrics['auc_std']:>4.3f}")

print(f"\n" + "="*70)
print("PER-CLASS RESULTS FOR HAR-ONLY")
print("="*70)

print(f"\n{'Class':<30s} {'Acc':>12s} {'Prec':>12s} {'Rec':>12s} {'F1':>12s} {'AUC':>12s}")
print("-" * 95)

for class_name in ACTIVITY_LABELS:
    metrics = aggregate_results['har']['per_class'][class_name]
    print(f"{class_name:<30s} "
          f"{metrics['accuracy_mean']*100:>5.2f}±{metrics['accuracy_std']*100:>4.2f}% "
          f"{metrics['precision_mean']:>5.3f}±{metrics['precision_std']:>4.3f} "
          f"{metrics['recall_mean']:>5.3f}±{metrics['recall_std']:>4.3f} "
          f"{metrics['f1_mean']:>5.3f}±{metrics['f1_std']:>4.3f} "
          f"{metrics['auc_mean']:>5.3f}±{metrics['auc_std']:>4.3f}")    
    
# Find best method
best_method = max(methods, key=lambda m: aggregate_results[m]['accuracy_mean'])
print(f"\nBest method: {method_names[best_method]} "
      f"({aggregate_results[best_method]['accuracy_mean']*100:.2f}% ± "
      f"{aggregate_results[best_method]['accuracy_std']*100:.2f}%)")

# Print per-class aggregate for best method
print(f"\n" + "="*70)
print(f"PER-CLASS RESULTS FOR BEST METHOD: {method_names[best_method].upper()}")
print("="*70)

print(f"\n{'Class':<30s} {'Acc':>12s} {'Prec':>12s} {'Rec':>12s} {'F1':>12s} {'AUC':>12s}")
print("-" * 95)

for class_name in ACTIVITY_LABELS:
    metrics = aggregate_results[best_method]['per_class'][class_name]
    print(f"{class_name:<30s} "
          f"{metrics['accuracy_mean']*100:>5.2f}±{metrics['accuracy_std']*100:>4.2f}% "
          f"{metrics['precision_mean']:>5.3f}±{metrics['precision_std']:>4.3f} "
          f"{metrics['recall_mean']:>5.3f}±{metrics['recall_std']:>4.3f} "
          f"{metrics['f1_mean']:>5.3f}±{metrics['f1_std']:>4.3f} "
          f"{metrics['auc_mean']:>5.3f}±{metrics['auc_std']:>4.3f}")

# Save final aggregate results
final_results = {
    'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    'experiment': 'Unified_Multimodal_EEG_HAR_Fusion_5Fold',
    'configuration': {
        'window_size_sec': WINDOW_SIZE_SEC,
        'overlap': OVERLAP,
        'eeg_fs': EEG_FS,
        'har_fs': HAR_FS,
        'n_folds': N_FOLDS,
        'random_seed': RANDOM_SEED
    },
    'activity_labels': ACTIVITY_LABELS,
    'aggregate_results': aggregate_results,
    'fold_results': all_fold_results,
    'statistical_tests': statistical_tests,
    'best_method': {
        'name': best_method,
        'display_name': method_names[best_method],
        'accuracy_mean': aggregate_results[best_method]['accuracy_mean'],
        'accuracy_std': aggregate_results[best_method]['accuracy_std'],
        'f1_macro_mean': aggregate_results[best_method]['f1_macro_mean'],
        'f1_macro_std': aggregate_results[best_method]['f1_macro_std']
    }
}

with open(os.path.join(OUTPUT_DIR, 'final_results.json'), 'w') as f:
    json.dump(final_results, f, indent=2)

# Create summary CSV for easy viewing
summary_data = []
for method in methods:
    row = {
        'Method': method_names[method],
        'Accuracy_Mean': aggregate_results[method]['accuracy_mean'] * 100,
        'Accuracy_Std': aggregate_results[method]['accuracy_std'] * 100,
        'F1_Mean': aggregate_results[method]['f1_macro_mean'],
        'F1_Std': aggregate_results[method]['f1_macro_std']
    }
    summary_data.append(row)

summary_df = pd.DataFrame(summary_data)
summary_df.to_csv(os.path.join(OUTPUT_DIR, 'method_comparison.csv'), index=False)

# Create robustness metrics CSV
robustness_data = []
for method in methods:
    row = {
        'Method': method_names[method],
        'Worst_Task_Acc_Mean': aggregate_results[method]['worst_task_accuracy_mean'] * 100,
        'Worst_Task_Acc_Std': aggregate_results[method]['worst_task_accuracy_std'] * 100,
        'Cross_Task_Std_Mean': aggregate_results[method]['cross_task_std_mean'] * 100,
        'Cross_Task_Std_Std': aggregate_results[method]['cross_task_std_std'] * 100
    }
    robustness_data.append(row)

robustness_df = pd.DataFrame(robustness_data)
robustness_df.to_csv(os.path.join(OUTPUT_DIR, 'robustness_metrics.csv'), index=False)

# Create statistical tests CSV
stats_data = []
for comparison in ['fusion_vs_eeg', 'fusion_vs_har', 'fusion_vs_best_single']:
    for metric in statistical_tests[comparison].keys():
        row = {
            'Comparison': comparison,
            'Metric': metric,
            'T_Statistic': statistical_tests[comparison][metric]['t_statistic'],
            'P_Value': statistical_tests[comparison][metric]['p_value'],
            'Significant': statistical_tests[comparison][metric]['p_value'] < 0.05
        }
        stats_data.append(row)

stats_df = pd.DataFrame(stats_data)
stats_df.to_csv(os.path.join(OUTPUT_DIR, 'statistical_tests.csv'), index=False)

# Create per-class summary CSV for best method
per_class_data = []
for class_name in ACTIVITY_LABELS:
    metrics = aggregate_results[best_method]['per_class'][class_name]
    row = {
        'Class': class_name,
        'Accuracy_Mean': metrics['accuracy_mean'] * 100,
        'Accuracy_Std': metrics['accuracy_std'] * 100,
        'Precision_Mean': metrics['precision_mean'],
        'Precision_Std': metrics['precision_std'],
        'Recall_Mean': metrics['recall_mean'],
        'Recall_Std': metrics['recall_std'],
        'F1_Mean': metrics['f1_mean'],
        'F1_Std': metrics['f1_std'],
        'AUC_Mean': metrics['auc_mean'],
        'AUC_Std': metrics['auc_std']
    }
    per_class_data.append(row)

per_class_df = pd.DataFrame(per_class_data)
per_class_df.to_csv(os.path.join(OUTPUT_DIR, f'{best_method}_per_class_results.csv'), index=False)

# Export EEG-only per-class results
eeg_per_class_data = []
for class_name in ACTIVITY_LABELS:
    metrics = aggregate_results['eeg']['per_class'][class_name]
    row = {
        'Class': class_name,
        'Accuracy_Mean': metrics['accuracy_mean'] * 100,
        'Accuracy_Std': metrics['accuracy_std'] * 100,
        'Precision_Mean': metrics['precision_mean'],
        'Precision_Std': metrics['precision_std'],
        'Recall_Mean': metrics['recall_mean'],
        'Recall_Std': metrics['recall_std'],
        'F1_Mean': metrics['f1_mean'],
        'F1_Std': metrics['f1_std'],
        'AUC_Mean': metrics['auc_mean'],
        'AUC_Std': metrics['auc_std']
    }
    eeg_per_class_data.append(row)

eeg_per_class_df = pd.DataFrame(eeg_per_class_data)
eeg_per_class_df.to_csv(os.path.join(OUTPUT_DIR, 'eeg_per_class_results.csv'), index=False)

# Export HAR-only per-class results
har_per_class_data = []
for class_name in ACTIVITY_LABELS:
    metrics = aggregate_results['har']['per_class'][class_name]
    row = {
        'Class': class_name,
        'Accuracy_Mean': metrics['accuracy_mean'] * 100,
        'Accuracy_Std': metrics['accuracy_std'] * 100,
        'Precision_Mean': metrics['precision_mean'],
        'Precision_Std': metrics['precision_std'],
        'Recall_Mean': metrics['recall_mean'],
        'Recall_Std': metrics['recall_std'],
        'F1_Mean': metrics['f1_mean'],
        'F1_Std': metrics['f1_std'],
        'AUC_Mean': metrics['auc_mean'],
        'AUC_Std': metrics['auc_std']
    }
    har_per_class_data.append(row)

har_per_class_df = pd.DataFrame(har_per_class_data)
har_per_class_df.to_csv(os.path.join(OUTPUT_DIR, 'har_per_class_results.csv'), index=False)

print(f"\nResults saved to: {OUTPUT_DIR}")
print(f"  - final_results.json: Complete results with all metrics")
print(f"  - method_comparison.csv: Method comparison summary")
print(f"  - robustness_metrics.csv: Worst-task accuracy and cross-task variance")
print(f"  - statistical_tests.csv: Paired t-test results")
print(f"  - eeg_per_class_results.csv: EEG-only per-class results")  
print(f"  - har_per_class_results.csv: HAR-only per-class results")  
print(f"  - {best_method}_per_class_results.csv: Per-class results for best method")
print(f"  - fold_X/: Individual fold results and confusion matrices")